# Deep Learning with Python

## 4.2 Evaluating machine-learning models

> 机器学习模型评估

[原文链接](https://livebook.manning.com/book/deep-learning-with-python/chapter-4/44)

### 训练集、验证集和测试集

- 训练集：用来学习参数（网络里各节点的权重）；
- 验证集：用来学习超参数（网络的权重，比如层数、层的大小这种）；
- 测试集：用来验证结果，要保证模型从未见过这些数据。

测试集必须是单独分出来的，训练集、测试集中不能和测试集有重合。

最好的做法是，先把所有数据分成训练集和测试集。然后从训练集里分一部分出来做验证集。

以下是几种选择验证集的方法：

#### 简单留出验证 （SIMPLE HOLD-OUT VALIDATION）

就是简单的从训练集里留出一部分来做验证集。

可用的数据多的时候才能用这个。不然数据少了，分出来的验证集就太小，不够一般，效果不好。

![简单留出验证的示意图](https://tva1.sinaimg.cn/large/007S8ZIlgy1ggkhlemz7cj31io0k0n02.jpg)

In [ ]:
# Hold-out validation

num_validation_samples = 10000

np.random.shuffle(data)    # 洗牌，打乱数据

validation_data = data[:num_validation_samples]    #定义验证集
data = data[num_validation_samples:]

training_data = data[:]    # 定义训练集

# 在训练集训练模型，在验证集评估
model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)

## 然后这里可以根据结果调整模型，
## 然后重新训练、评估，然后再次调整...

# 在调整好超参数之后，用除了测试集的所有数据来训练最终模型
model = get_model()
model.train(np.concatenate([training_data, validation_data]))

# 用测试集来评估最终模型
test_score = model.evaluate(test_data)

#### K折验证 (K-FOLD VALIDATION)

这个方法是把数据等分成 K 份。对每个部分 i，在剩下的 K-1 个部分里训练，在 i 上验证评估。最终验证的结果取 K 次的验证值的平均。

这种方法对不同的训练、验证集划分对结果影响比较大时会很有效（比如数据比较少的时候）

![K折验证示意图](https://tva1.sinaimg.cn/large/007S8ZIlgy1ggki8sc8x0j31os0u0q90.jpg)

emmm，我觉得这个图有点问题，应该除了那些灰色的是 Validation，白的应该都是 Training。（中文译本上就是这种）

In [ ]:
# K-fold cross-validation

k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    # 选择验证集
    validation_data = data[num_validation_samples * fold:
                           num_validation_samples * (fold + 1)]
    # 用剩下的做训练集
    training_data = data[:num_validation_samples * fold] + 
                    data[num_validation_samples * (fold + 1):]
    
    model = get_model()    # 注意是用个全新的模型
    
    model.train(training_data)
    
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)

# 总的验证值是所有的平均
validation_score = np.average(validation_scores)

# 然后根据结果做各种超参数调整啦

# 最后在除了测试集的所有数据上训练
model = get_model()
model.train(data)
test_score = model.evaluate(test_data)

#### 带有打乱数据的重复K折验证 (ITERATED K-FOLD VALIDATION WITH SHUFFLING)

就是重复跑 P 次 K折验证，每次开始前洗牌。

这个是数据比较少，又要求尽可能精确的时候用的。要跑 P*K 次，所以比较耗时。

书上没给这个的代码，就是在 K折 的基础上再加一层循环：

In [ ]:
# ITERATED K-FOLD VALIDATION WITH SHUFFLING

p = 10

k = 4
num_validation_samples = len(data) // k

total_validation_scores = []

for i in range(p):
    np.random.shuffle(data)
    
    validation_scores = []
    for fold in range(k):
        # TODO: K折验证的那堆代码
    
    validation_score = np.average(validation_scores)
    total_validation_scores.append(validation_score)
    
# 总的验证值是所有的平均
validation_score = np.average(total_validation_scores)

# 然后根据结果做各种超参数调整啦

# 最后在除了测试集的所有数据上训练
model = get_model()
model.train(data)
test_score = model.evaluate(test_data)

### 划分时的注意事项

- **数据代表性**：训练集和测试集都要可以代表所有数据。比如做数字识别，不能训练集里只有 0～7，测试集里全是 8～9。做这种之前要把所有数据随机洗牌打乱，然后再分训练集和测试集。
- **时间箭头**：如果是那种做跟时间有关的预测（给过去的，预测未来的），开始之前**不要打乱数据**，要保持数据的时间顺序（打乱了会时间泄露，就是模型从“未来”学习了），并且测试集的数据要晚于训练集。
- **数据冗余**：如果数据有重复，随机打乱之后，同样的数据就可能同时出现在训练、验证、测试集里了。这种情况会影响结果，训练和验证集不能有交集。